In [ ]:

def subreddit_Puller(sub,sql_db,start_date,end_date):
	'''
	(str,str) -> None
	
	Takes sub input (str) that represents the subreddit that will be parsed 
	and have all its submissions extracted and placed into NBA_submissions table.
	Also gets unique redditor list in new table.
	
	'''
	import sqlite3
	import praw
	from time import gmtime,strftime
	
	conn = sqlite3.connect(sql_db)
	cur = conn.cursor()
	
	#checks if table exists and deletes and creates new one in DB
	#.execute('DROP TABLE IF EXISTS {}'.format(sub,))
	cur.execute('''
	CREATE TABLE IF NOT EXISTS NBA_submissions(
		id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
		submission_title TEXT,
		sub_red_id TEXT,
		submission_score INTEGER,
		num_comments INTEGER,
		date_created DATE,
		submission_url TEXT,
		reddit_url TEXT,
		team_id INTEGER,
		redditor_auth_id INTEGER)''')
	
	#gets unique team_id value from NBA_subs table (foreign key)
	cur.execute('''
	SELECT id FROM NBA_subs WHERE sub_name = ?
	''',(sub,))
	team_id = cur.fetchone()[0]
	#establishes praw reddit api object and accesses subreddit object
	reddit = praw.Reddit('red_bot1')
	subreddit = reddit.subreddit(sub)
	
	#pulls relavant submission data through praw api
	for submission in subreddit.submissions(start_date,end_date):
		submission_score = submission.score
		submission_title = str(submission.title)
		num_comments = submission.num_comments
		date_created = strftime("%Y-%m-%d %H:%M:%S UTC",gmtime(submission.created_utc))
		redditor_auth = str(submission.author)
		submission_url = submission.url
		reddit_url = "www.reddit.com" + str(submission.permalink)
		sub_red_id = submission.id
		
		if submission_title is None or submission_score is None : continue
		
		#checks to see if submission is already recorded, if so skips it
		cur.execute("SELECT * FROM NBA_submissions WHERE sub_red_id = ?", (sub_red_id,))
		submission_already_Present = cur.fetchone()
		if submission_already_Present is not None : 
			print("SKIP IN {}-{}".format(sub,date_created))
			continue
		
		
		#sanity check
		print('Submission: {},{},{},{},{},{},{}'.format(redditor_auth,submission_title,submission_score,num_comments,date_created,submission_url,reddit_url))
		
		#checks to see if redditor already stored in NBA_redditor_list table and if not adds them
		cur.execute("SELECT * FROM NBA_redditor_list WHERE redditor_auth = ?", (redditor_auth,))
		redditor_already_Present = cur.fetchone()
		if redditor_already_Present is None:
			cur.execute("INSERT INTO NBA_redditor_list (redditor_auth) VALUES (?)",(redditor_auth,))
			cur.execute("SELECT * FROM NBA_redditor_list WHERE redditor_auth = (?)",(redditor_auth,))
			redditor_auth_id = cur.fetchone()[0]
		else:
			cur.execute("SELECT * FROM NBA_redditor_list WHERE redditor_auth = (?)",(redditor_auth,))
			redditor_auth_id = cur.fetchone()[0]
			
			
		cur.execute("INSERT OR IGNORE INTO NBA_submissions (submission_title,sub_red_id,submission_score,num_comments,date_created,submission_url,reddit_url,team_id,redditor_auth_id) VALUES (?,?,?,?,?,?,?,?,?)", (submission_title,sub_red_id,submission_score,num_comments,date_created,submission_url,reddit_url,team_id,redditor_auth_id))
		
	conn.commit()
	print("Done extracting subreddit {}".format(sub))
	return None

	
import sqlite3
sql_db = 'reddit_db.sqlite'	
subs = []

conn = sqlite3.connect(sql_db)
cur = conn.cursor()

#delete's present NBA_redditor_list table if it exists and creates new one before data is loaded into it
#cur.execute('DROP TABLE IF EXISTS NBA_redditor_list')
cur.execute('''
	CREATE TABLE IF NOT EXISTS NBA_redditor_list(
		id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
		redditor_auth TEXT
	)''')


#pulls nba subreddit names from reddit_db DB. This data was generated by the accompanying python script 
cur.execute("SELECT sub_name FROM NBA_subs")
for team in cur.fetchall():
	subs.append(team[0])
print(subs)

#We pass the date ranges we want in here
#[(1/1/2012,1/1/2013),(1/1/2013,1/1/2014),(1/1/2014,1/1/2015),(1/1/2015,1/1/2016),(1/1/2016,1/1/2017),(1/1/2017, to present)]
#date_ranges = [(1325376000,1356998400),(1356998400,1388534400),(1388534400,1420070400),(1420070400,1451606400),(1451606400,1483228800),(1483228800,None)]
date_ranges = [(1420070400,1451606400),(1451606400,1483228800),(1483228800,None)]
for date_tup in date_ranges:
	[subreddit_Puller(x,sql_db,date_tup[0],date_tup[1]) for x in subs]


